In [1]:
!pip install jyutping
!pip install pypinyin
!pip install opencc-python-reimplemented

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for jyutping: filename=jyutping-0.3.4-py2.py3-none-any.whl size=39219 sha256=e0ec8c1d45a0986834186433d999dd8d39029d6bd6f1fe6827a04847aa84cf92
  Stored in directory: /home/codespace/.cache/pip/wheels/7f/98/14/ec11d92e946d436f3762d4104b088672f3b7ac02059cc67918
Successfully built jyutping

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 18.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
# list all GB2312 sections first

# start with the most common chinese character: 你好

"你好".encode("gb2312"), "你".encode("gb2312"), "好".encode("gb2312")

(b'\xc4\xe3\xba\xc3', b'\xc4\xe3', b'\xba\xc3')

In [43]:
# try bisect the first gb2312 character

def get_num(chr: str, encoding="gb2312", byteorder_little=False):
    decode_result = chr.encode(encoding)
    length = len(decode_result)
    num = 0

    if byteorder_little:
        decode_result = decode_result[::-1]

    for bytevalue in decode_result:
        num *= 256
        num += bytevalue
    return num

import math

def get_str(num: int, encoding="gb2312", byteorder_little=False):
    if num < 0:
        raise ValueError("no negative num allowed")
    if num == 0:
        return b'\x00'.decode(encoding=encoding)
    len_of_num = math.floor((len(hex(num)) - 2) / 2)
    return num.to_bytes(len_of_num, 'little' if byteorder_little else 'big', signed=False) \
    .decode(encoding=encoding)


get_str(get_num("你")), get_num("你", byteorder_little=True)


('你', 58308)

In [41]:
hex(31415926)

'0x1df5e76'

In [19]:
import struct
struct.unpack("H", "你".encode("gb2312"))[0]

58308

In [24]:
# prepare bisecting; first requires a judge to tell whether something is chinese or not
!pip install chinese

# get ce-dict in chinese: not obvious

import chinese
!pip install pycccedict

from pycccedict.cccedict import CcCedict

cccedict = CcCedict()
cccedict.get_entry('你')


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


{'traditional': '妳',
 'simplified': '你',
 'pinyin': 'ni3',
 'definitions': ['you (Note: In Taiwan, 妳 is used to address females, but in mainland China, it is not commonly used. Instead, 你 is used to address both males and females.)']}

In [30]:
print(cccedict.get_entry('2'))

None


In [60]:
def is_chinese(num, verbose=False):
    # try decode
    try:
        character = get_str(num)
        item = cccedict.get_entry(character)
        return item is not None
    except Exception as e:
        if verbose:
            import sys
            print(f"{type(e)} for {num}", file=sys.stderr, flush=True)
        return False

In [61]:
# start bisecting

def bisect(lo, hi, judge):
    while lo < hi:
        mid = (lo + hi) // 2
        if judge(mid):
            hi = mid
        else:
            lo = mid + 1
    return lo, hi

bisect(0, get_num("你"), is_chinese)
    

(47777, 47777)

In [62]:
bisect(47777, 65536, is_chinese)

(57761, 57761)

In [63]:
gb2312_range = (47777, 57761)
get_str(gb2312_range[0]), get_str(gb2312_range[1])

('骸', '帷')

In [70]:
# seems that bisect doesn't work

for i in range(65536):
    if is_chinese(i):
        print(i, get_str(i), cccedict.get_entry(get_str(i))['pinyin'])

37 % pa1
65 A A
66 B bi1
80 P P
81 Q Q
84 T T
119 w w
13123 3C san1 C
13136 3P san1 P
13137 3Q san1 Q
14392 88 ba1 ba1
16973 BM B M
17232 CP C P
18768 IP I P
19779 MC M C
19798 MV M V
20039 NG N G
20307 OS O S
20545 PA P A
20555 PK P K
20559 PO po1
20565 PU P U
21569 TA ta1
41385 々 xx5
41456 ○ ling2
41461 □ ging1
45217 啊 a5
45218 阿 e1
45219 埃 ai1
45220 挨 ai2
45221 哎 ai1
45222 唉 ai4
45223 哀 ai1
45224 皑 ai2
45225 癌 ai2
45226 蔼 ai3
45227 矮 ai3
45228 艾 yi4
45229 碍 ai4
45230 爱 ai4
45231 隘 ai4
45232 鞍 an1
45233 氨 an1
45234 安 an1
45235 俺 an3
45236 按 an4
45237 暗 an4
45238 岸 an4
45239 胺 an4
45240 案 an4
45241 肮 ang1
45242 昂 ang2
45243 盎 ang4
45244 凹 wa1
45245 敖 ao2
45246 熬 ao2
45247 翱 ao2
45248 袄 ao3
45249 傲 ao4
45250 奥 ao4
45251 懊 ao4
45252 澳 ao4
45253 芭 ba1
45254 捌 ba1
45255 扒 pa2
45256 叭 ba1
45257 吧 bia1
45258 笆 ba1
45259 八 ba1
45260 疤 ba1
45261 巴 ba1
45262 拔 ba2
45263 跋 ba2
45264 靶 ba3
45265 把 ba4
45266 耙 pa2
45267 坝 ba4
45268 霸 ba4
45269 罢 ba5
45270 爸 ba4
45271 白 bai2
45272 柏 bai3
45273 百 b

In [78]:
section_1 = [45217,55290]
section_2 = [55290,63485]

In [ ]:
# check the sections

exception_character_ranges_for_section_1 = []

for i in range(section_1[0], section_1[1]):
    if not is_chinese(i):
        # check last range
        if exception_character_ranges_for_section_1 and exception_character_ranges_for_section_1[-1][1] == i:
            exception_character_ranges_for_section_1[-1][1] += 1
        else:
            exception_character_ranges_for_section_1.append([i,i+1])
exception_character_ranges_for_section_1


[[45311, 45473],
 [45567, 45729],
 [45823, 45985],
 [46079, 46241],
 [46335, 46497],
 [46591, 46753],
 [46847, 47009],
 [47103, 47265],
 [47359, 47521],
 [47615, 47777],
 [47871, 48033],
 [48127, 48289],
 [48383, 48545],
 [48639, 48801],
 [48895, 49057],
 [49151, 49313],
 [49407, 49569],
 [49663, 49825],
 [49919, 50081],
 [50175, 50337],
 [50431, 50593],
 [50687, 50849],
 [50943, 51105],
 [51199, 51361],
 [51455, 51617],
 [51711, 51873],
 [51967, 52129],
 [52223, 52385],
 [52479, 52641],
 [52735, 52897],
 [52991, 53153],
 [53247, 53409],
 [53503, 53665],
 [53759, 53921],
 [54015, 54177],
 [54271, 54433],
 [54527, 54689],
 [54783, 54945],
 [55039, 55201]]